In [1]:
import os
import glob
import pydicom
import numpy as np
import cv2
from scipy.ndimage import zoom
from tqdm import tqdm
import cv2
import pandas as pd
import math
import torch
import torch.nn as nn


In [2]:
def load_slices(patient, scan_type='T2w'):
    sorted_imgs = sorted(glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/'+patient+'/'+scan_type+'/*.dcm'))
    slices = [pydicom.read_file(s) for s in sorted_imgs]
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(np.int16)
    #image[image == -2000] = 0
    return np.array(image, dtype=np.int16)

def remove_blanks(hu_images):
    blanked_images = []
    for i in range(hu_images.shape[0]):
        if np.min(hu_images[i]) != np.max(hu_images[i]):
            blanked_images.append(hu_images[i])
    return np.array(blanked_images, dtype=np.int16)

def cropped_images(images):
    '''
    Input images numpy array (D, H, W) (D = Depth)
    '''
    min_1=np.array(np.nonzero(images)).min(axis=1)
    max_1=np.array(np.nonzero(images)).max(axis=1)
    min_crop = min(min_1[1], min_1[2])
    max_crop = max(max_1[1], max_1[2])
    return images[min_1[0]:max_1[0], min_crop:max_crop,min_crop:max_crop]

def siz(img, deth=32):
    desired_depth = deth
    current_depth = img.shape[0]
    depth = current_depth / desired_depth
    depth_factor = 1 / depth
    img_new = zoom(img, (depth_factor, 1, 1), mode='nearest')
    return img_new

def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [3]:
class ConvSection(nn.Module):
    """
    A convolution section
    """
    def __init__(self, in_channels, mid_channels, resize_conv=None, stride=1):
        super(ConvSection, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm3d(mid_channels)

        self.conv2 = nn.Conv3d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(mid_channels)

        self.conv3 = nn.Conv3d(mid_channels, mid_channels*4, kernel_size=1, padding=0, stride=1, bias=False)
        self.bn3 = nn.BatchNorm3d(mid_channels*4)
        
        self.relu = nn.ReLU()
        self.resize_conv = resize_conv
        self.stride = stride

    def forward(self, x):
        idn = x.clone()
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        
        if self.resize_conv is not None:
            idn = self.resize_conv(idn)
        
        x += idn
        x = self.relu(x)
        return x

class Resnet503D(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Resnet503D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU()
        self.mpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)

        self.apool = nn.AdaptiveAvgPool3d((1,1,1)) #TODO: set avgadaptive pool output
        
        self.layer_channel = 64
        self.conv2_x = self.convLayer(3, 64, 1)
        self.conv3_x = self.convLayer(4, 128, 2)
        self.conv4_x = self.convLayer(6, 256, 2)
        self.conv5_x = self.convLayer(3, 512, 2)
        
        self.fc = nn.Linear(512*4, num_classes)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(262144, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dp1 = nn.Dropout(0.4)
        self.dp2 = nn.Dropout(0.4)
        
    def convLayer(self, num_blocks, mid_channels, stride):
        layers = []
        resize_conv = None

        if stride != 1 or self.layer_channel != mid_channels*4:
            resize_conv = nn.Sequential(
                nn.Conv3d(self.layer_channel, mid_channels*4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(mid_channels*4),
            )
        layers.append(ConvSection(self.layer_channel, mid_channels, resize_conv, stride))
        self.layer_channel = mid_channels*4
        for i in range(num_blocks - 1):
            layers.append(ConvSection(self.layer_channel, mid_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.mpool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.apool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        #x = self.flatten(x)
        #x = self.fc1(x)
        #x = self.relu(x)
        #x = self.dp1(x)
        #x = self.fc2(x)
        #x = self.relu(x)
        #x = self.dp2(x)
        #x = self.fc3(x)
        # with logits output
        return x

In [4]:
model_1 = Resnet503D(in_channels=1, num_classes=1)
print("==> loading checkpoint")
checkpoint = torch.load('../input/edemarsna/T2w-R50-Alone-0.6486111111111111.pth.tar')
model_1.load_state_dict(checkpoint['state_dic'])
model_1.to(device='cuda')
model_1.eval()

model_3 = Resnet503D(in_channels=1, num_classes=1)
print("==> loading checkpoint")
checkpoint = torch.load('../input/edemarsna/T2w-R50-final-0.6476813721656799.pth.tar')
model_3.load_state_dict(checkpoint['state_dic'])
model_3.to(device='cuda')
model_3.eval()

model_4 = Resnet503D(in_channels=1, num_classes=1)
print("==> loading checkpoint")
checkpoint = torch.load('../input/edemarsna/T2w-R50-final-0.6517584300041199.pth.tar')
model_4.load_state_dict(checkpoint['state_dic'])
model_4.to(device='cuda')
model_4.eval()

model_5 = Resnet503D(in_channels=1, num_classes=1)
print("==> loading checkpoint")
checkpoint = torch.load('../input/edemarsna/T2w-R50-final-0.6553582501411438.pth.tar')
model_5.load_state_dict(checkpoint['state_dic'])
model_5.to(device='cuda')
model_5.eval()

model_6 = Resnet503D(in_channels=1, num_classes=1)
print("==> loading checkpoint")
checkpoint = torch.load('../input/edemarsna/T2w-R50-final-0.6569597268104553.pth.tar')
model_6.load_state_dict(checkpoint['state_dic'])
model_6.to(device='cuda')
model_6.eval()

==> loading checkpoint
==> loading checkpoint
==> loading checkpoint
==> loading checkpoint
==> loading checkpoint


Resnet503D(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (mpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (apool): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (conv2_x): Sequential(
    (0): ConvSection(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (resize_conv): Sequentia

In [5]:
# Prediction loop
patients = os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/')
output = []
for patient in tqdm(patients):
    if patient == '00123' or patient == '00109' or patient == '00709':
        print(f'removed {patient}')
        continue
    
    slices = load_slices(patient)
    images = get_pixels_hu(slices)
    images = remove_blanks(images)
    if images.shape[0] < 1:
        print(f'removed {patient}')
        continue
    images = cropped_images(images)
    im = []
    for j in range(images.shape[0]):
        im.append(cv2.resize(images[j], (256, 256)))
    images = np.array(im)
    images = siz(images, 64)
    images = normalize(images)
    images = torch.from_numpy(images)
    images = torch.reshape(images, (1, 1, 64, 256, 256))
    with torch.no_grad():
        images = images.to(device='cuda', dtype=torch.float)
        pred_1 = model_1(images)
        pred_3 = model_3(images)
        pred_4 = model_4(images)
        pred_5 = model_5(images)
        pred_6 = model_6(images)
    p_1 = torch.sigmoid(pred_1)[0].item()
    p_3 = torch.sigmoid(pred_3)[0].item()
    p_4 = torch.sigmoid(pred_4)[0].item()
    p_5 = torch.sigmoid(pred_5)[0].item()
    p_6 = torch.sigmoid(pred_6)[0].item()
    prediction = p_1 + p_3 + p_4 + p_5 + p_6
    prediction = prediction / 5
    output.append([patient, prediction])
    
df = pd.DataFrame(output, columns = ['BraTS21ID', 'MGMT_value'])
df = df.set_index('BraTS21ID')
df.to_csv('submission.csv')

100%|██████████| 87/87 [05:41<00:00,  3.92s/it]


In [6]:
df

,MGMT_value
BraTS21ID,
00114,0.333541
00013,0.698057
00821,0.420736
00644,0.704995
00699,0.715230
...,...
00474,0.495502
00174,0.455157
00119,0.511923
